#### @prompt example

In [34]:
from openbb import obb
import os
obb.account.login(pat=os.environ['PAT_KEY'])

In [35]:
obb.news.world(provider='fmp')

UnauthorizedError: 
[Error] -> Unauthorized FMP request -> Limit Reach . Please upgrade your plan or visit our documentation for more details at https://site.financialmodelingprep.com/

In [ ]:
from magentic import prompt
from openbb import obb

obb.ac


@prompt("Explain like I'm five this financial concept: {concept}")
def explain(concept: str) -> str: ...


explain("Subprime mortgage crisis")

AttributeError: 'App' object has no attribute 'ac'

In [36]:
from magentic import prompt
from pydantic import BaseModel


class Portfolio(BaseModel):
    equity_etf_pct: float
    bond_etf_pc: float
    crypto_etf_pc: float
    commodities_pc: float
    reasoning: str


@prompt("Create a strong portfolio of {size} allocation size.")
def create_portfolio(size: str) -> Portfolio: ...


portfolio = create_portfolio("$50,000")

In [37]:
print(portfolio)

equity_etf_pct=60.0 bond_etf_pc=25.0 crypto_etf_pc=5.0 commodities_pc=10.0 reasoning='The portfolio is designed with a moderate risk tolerance in mind to balance growth and stability. 60% is allocated to equity ETFs to capitalize on the growth potential of the stock market, which historically offers higher returns over the long term. 25% is allocated to bond ETFs to provide stability and income, serving as a hedge against stock market volatility. 5% is allocated to crypto ETFs, offering exposure to potentially high-reward assets, although in a smaller, more controlled portion due to their inherent volatility. 10% goes into commodities, which can act as a safeguard against inflation and add diversification benefits. This diversification aligns with creating a strong portfolio that manages risk while harnessing growth opportunities in varied market conditions.'


#### @chatprompt decorator

In [38]:
from magentic import chatprompt, AssistantMessage, SystemMessage, UserMessage
from pydantic import BaseModel


class Quote(BaseModel):
    quote: str
    person: str


@chatprompt(
    SystemMessage("You are an avid reader of financial literature."),
    UserMessage("What is your favorite quote from Warren Buffet?"),
    AssistantMessage(
        Quote(
            quote="Price is what you pay; value is what you get.",
            person="Warren Buffet",
        )
    ),
    UserMessage("What is your favorite quote from {person}?"),
)
def get_finance_quote(person: str) -> Quote: ...


get_finance_quote("Charlie Munger")

Quote(quote='The big money is not in the buying or selling, but in the waiting.', person='Charlie Munger')

#### Function calling

In [39]:
import os
import requests
from magentic import prompt, FunctionCall
import logging

FMP_KEY = os.getenv("FMP_KEY")


def get_daily_price(ticker: str, api_key: str = FMP_KEY) -> dict:
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={ticker}&apikey={api_key}"
    r = requests.get(url)
    data = r.json()
    return data["Time Series (Daily)"]

def get_quote(query : str)-> dict:

    ticker = 'MO'
    logging.info('Querying for {ticker}')

    url = f"https://financialmodelingprep.com/api/v3/quote/{ticker}?apikey={os.environ['FMP_KEY']}"
    response = requests.get(url).json()[0] 
    logging.info(f'We got {response}')
    return response

def get_news()-> dict :
    return obb.news.world(provider='fmp')

def get_aggressive_smallcaps() -> list :
    return obb.equity.discovery.aggressive_small_caps(sort='desc')

def get_sectors_performance() -> list:
    return obb.equity.compare.groups(group='sector', metric='performance', provider='finviz')

def get_undervalued_growth() -> list:
    return obb.equity.discovery.undervalued_growth(provider='yfinance')

@prompt(
    "Use the appropriate search function to answer: {question}",
    functions=[get_daily_price, get_quote, get_news, get_aggressive_smallcaps, get_sectors_performance, get_undervalued_growth],
)
def perform_search(question: str) -> FunctionCall[str]: ...


output = perform_search("What is the latest quote for  AAPL?")

In [40]:
print(output)

FunctionCall(<function get_quote at 0x714f6d09d620>, 'AAPL')


In [41]:
output()

{'symbol': 'MO',
 'name': 'Altria Group, Inc.',
 'price': 57.79,
 'changesPercentage': 1.36818,
 'change': 0.78,
 'dayLow': 56.81,
 'dayHigh': 58.5484,
 'yearHigh': 58.59,
 'yearLow': 40.65,
 'marketCap': 97703241400,
 'priceAvg50': 53.208,
 'priceAvg200': 51.36615,
 'exchange': 'NYSE',
 'volume': 12634604,
 'avgVolume': 8198878,
 'open': 56.86,
 'previousClose': 57.01,
 'eps': 6.54,
 'pe': 8.84,
 'earningsAnnouncement': '2025-04-29T12:30:00.000+0000',
 'sharesOutstanding': 1690660000,
 'timestamp': 1741381202}

In [42]:
output = perform_search('Please give me a list of latest aggressive smallcaps')
output

FunctionCall(<function get_aggressive_smallcaps at 0x714f6d0bf420>, )

In [44]:
output().to_df()

,symbol,name,price,change,percent_change,volume,open,high,low,previous_close,...,book_value,price_to_book,eps_ttm,eps_forward,pe_forward,dividend_yield,exchange,exchange_timezone,earnings_date,currency
0,MLNK,"MeridianLink, Inc.",21.49,3.060000,0.166034,731416,19.500,21.8700,19.110,18.43,...,5.621,3.823163,-0.39,0.50,42.980000,0.00000,NYQ,America/New_York,2025-03-06 16:00:00-05:00,USD
1,BYON,"Beyond, Inc.",6.14,0.600000,0.108303,3349951,5.550,6.4250,5.585,5.54,...,3.066,2.002609,-5.56,-1.77,-3.468926,0.00000,NYQ,America/New_York,2025-02-24 16:42:56-05:00,USD
2,VEL,"Velocity Financial, Inc.",20.48,1.600000,0.084746,220982,20.020,20.7900,19.940,18.88,...,15.410,1.329007,1.84,2.13,9.615023,0.00000,NYQ,America/New_York,2025-03-06 16:10:00-05:00,USD
3,BMBL,Bumble Inc.,5.08,0.350000,0.073996,5159961,4.640,5.1750,4.610,4.73,...,7.698,0.659912,-4.61,0.76,6.684210,0.00000,NMS,America/New_York,2025-02-18 16:05:00-05:00,USD
4,AZTA,"Azenta, Inc.",43.68,2.570000,0.062515,810101,41.735,44.0400,40.395,41.11,...,37.621,1.161054,-3.17,0.42,104.000010,0.00000,NMS,America/New_York,2025-02-05 07:00:19-05:00,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,MERC,Mercer International Inc.,7.01,-0.370000,-0.050135,534298,7.390,7.4300,6.930,7.38,...,6.429,1.090372,-1.27,-0.19,-36.894737,0.04065,NMS,America/New_York,2025-02-20 16:30:27-05:00,USD
241,SNDX,"Syndax Pharmaceuticals, Inc.",13.42,-0.870000,-0.060882,2507559,14.200,14.3900,13.240,14.29,...,3.362,3.991672,-3.72,-3.54,-3.790961,0.00000,NMS,America/New_York,2025-03-03 07:01:00-05:00,USD
242,TBI,"TrueBlue, Inc.",5.70,-0.370000,-0.060956,198867,6.040,6.0400,5.700,6.07,...,10.658,0.534810,-4.17,-0.11,-51.818180,0.00000,NYQ,America/New_York,2025-02-19 16:05:00-05:00,USD
243,PHAT,"Phathom Pharmaceuticals, Inc.",5.06,-0.840000,-0.142373,1796298,5.770,5.7700,5.015,5.90,...,-3.701,-1.367198,-5.29,-4.00,-1.265000,0.00000,NMS,America/New_York,2025-03-06 08:00:02-05:00,USD


#### Prompt Chains

In [ ]:
import csv
from magentic import prompt_chain


def get_earnings_calendar(ticker: str, api_key: str = FMP_KEY) -> list:
    url = f"https://www.alphavantage.co/query?function=EARNINGS_CALENDAR&symbol={ticker}&horizon=12month&apikey={api_key}"
    with requests.Session() as s:
        download = s.get(url)
        decoded_content = download.content.decode("utf-8")
        cr = csv.reader(decoded_content.splitlines(), delimiter=",")
        my_list = list(cr)
    return my_list


@prompt_chain(
    "What's {ticker} expected earnings dates for the next 12 months?",
    functions=[get_earnings_calendar],
)
def get_earnings(ticker: str) -> str: ...


get_earnings("IBM")

#### Streaming Response

In [ ]:
from magentic import StreamedStr


@prompt("Explain to me {term} in a way a 5-year-old would understand.")
def describe_finance_term(term: str) -> StreamedStr: ...


# Print the chunks while they are being received
for chunk in describe_finance_term("liquidity"):
    print(chunk, end="")

#### Streaming Structured Outputs

In [ ]:
from collections.abc import Iterable
from time import time


class Portfolio(BaseModel):
    equity_etf_pct: float
    bond_etf_pc: float
    crypto_etf_pc: float
    commodities_pc: float
    reasoning: str


@prompt("Create {n_portfolio} portfolios with varying deegress of risk apetite.")
def create_portfolios(n_portfolio: int) -> Iterable[Portfolio]: ...


start_time = time()
for portfolio in create_portfolios(3):
    print(f"{time() - start_time:.2f}s : {portfolio}")

#### Asynchronous Streaming

In [ ]:
import asyncio
from typing import AsyncIterable


@prompt("List three high-growth stocks.")
async def iter_growth_stocks() -> AsyncIterable[str]: ...


@prompt("Tell me more about {stock_symbol}")
async def tell_me_more_about(stock_symbol: str) -> str: ...


start_time = time()
tasks = []
async for stock in await iter_growth_stocks():
    # Use asyncio.create_task to schedule the coroutine for execution before awaiting it
    # This way descriptions will start being generated while the list of stocks is still being generated
    task = asyncio.create_task(tell_me_more_about(stock))
    tasks.append(task)

descriptions = await asyncio.gather(*tasks)

for desc in descriptions:
    print(desc)